In [ ]:
import pandas as pd

#### Le but 

En repartant du [travail de siretisation des mairies de l'annuaire du service public](https://pad.incubateur.net/u5jVO0ekT9SvUNB85Gr8Ng#), croiser avec les données data.inclusion sur la base du siret.



In [ ]:
data_inclusion_df = pd.read_csv(
    "https://www.data.gouv.fr/fr/datasets/r/fd4cb3ef-5c31-4c99-92fe-2cd8016c0ca5",
    dtype=str,
)

In [ ]:
# on isole les données provenant de l'annuaire
data_inclusion_annuaire_df = data_inclusion_df.query("source == 'etab_publics'")

# et on les retire les données
data_inclusion_df = data_inclusion_df.query("source != 'etab_publics'")

# on retire les données sans sirets
data_inclusion_df = data_inclusion_df[data_inclusion_df.siret.notna()]

In [ ]:
# fichier issu du travail de siretisation des mairies
annuaire_mairies_df = pd.read_csv(
    "https://data-inclusion-lake.s3.fr-par.scw.cloud/extra/base-donnees-locales-mairies-sirets-20230109.csv",
    sep="|",
    dtype=str,
)

# on retire les données sans sirets
annuaire_mairies_df = annuaire_mairies_df[annuaire_mairies_df.siret.notna()]

In [ ]:
# la jointure via siret
results_df = pd.merge(
    annuaire_mairies_df,
    data_inclusion_df,
    how="left",
    on="siret",
    suffixes=("_annuaire", ""),
)

results_df = results_df.assign(siret_annuaire=results_df.siret)

In [ ]:
# on ne garde que les matchs
results_df = results_df[results_df.source.notna()]

In [ ]:
results_df.source.value_counts().to_frame()

In [ ]:
# on isole les données de l'annuaire qui ont matchés pour les réintroduire
data_inclusion_annuaire_df = data_inclusion_annuaire_df[data_inclusion_annuaire_df.id.isin(results_df.id_annuaire)]

results_annuaire_df = pd.merge(
    annuaire_mairies_df,
    data_inclusion_annuaire_df,
    how="inner",
    on="id",
    suffixes=("_annuaire", ""),
)

results_annuaire_df = results_annuaire_df.assign(id_annuaire=results_annuaire_df.id)

In [ ]:
results_df = pd.concat([results_df, results_annuaire_df])

In [ ]:
# on groupe par identifiant dans l'annuaire et source dans data.inclusion afin de distinguer les doublons au sein d'une source
final_df = results_df.assign(rang=results_df.groupby(["id_annuaire", "source"])["id_annuaire"].rank(method="first").astype(int))

In [ ]:
# on pivote la table
final_df = final_df.pivot(index=["id_annuaire", "nom_annuaire", "siret_annuaire", "lien", "rang"], columns="source")

In [ ]:
import openpyxl.utils

with pd.ExcelWriter(
    "/tmp/base-donnees-locales-mairies-croisements-20230202.xlsx",
    engine="openpyxl",
) as writer:
    for schema_column_name, df in final_df.groupby(axis=1, level=0):
        df.to_excel(writer, sheet_name=schema_column_name)

        # adjust column width
        worksheet = writer.sheets[schema_column_name]
        for column_cells in worksheet.columns:
            length = min(max(len(str(cell.value) if cell.value else "") for cell in column_cells), 50)
            worksheet.column_dimensions[openpyxl.utils.get_column_letter(column_cells[0].column)].width = length
